In [51]:
#CODE TO PRODUCE REPLICAS OF JAME'S CHARTS
import pandas as pd
import numpy as np

In [54]:
#STEP ONE: read and clean the three sets of data (CBO21, CBO24, and OMB's outlays)

#CBO24
years = [i for i in range(2024, 2024+11)] 
cbo_24 = pd.read_excel('../data/cbo-2024-02-Spending-Projections.xlsx', header=7, sheet_name=1)
cbo_24 = cbo_24[[col for col in cbo_24.columns if col not in years]] #remove the BA cols
cbo_24.columns = cbo_24.columns.str.replace('.1', '') #clean the OT cols
cbo_24 = cbo_24.drop(columns=['Unnamed: 9', "Unnamed: 21", "Off-Budget?"]).drop(index=0).drop(cbo_24.index[-2:]) #drop the weird empty rows and the empty cols and offbud col
#cbo_24['25-34'] = cbo_24.loc[:, '2025':'2034'].sum(axis=1) #sum the ots over the 10yr window
cbo_24["Agency"] = cbo_24["Agency"].str.strip()
years = [str(i) for i in years]

#cbo_24_agencies = cbo_24[['Agency'] + years + ["25-34"]].groupby("Agency").sum().reset_index() #get sum of ots by agency
cbo_24_agencies = cbo_24[['Agency'] + years].groupby("Agency").sum().reset_index() #get sum of ots by agency
cbo_24_agencies['source'] = 'CBO24'
cbo_24_agencies[[str(year) for year in range(2018,2024)]] = np.nan
cbo_24_agencies.head(3)

,Agency,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,source,2018,2019,2020,2021,2022,2023
0,400 Years of African-American History Commission,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,CBO24,NaN,NaN,NaN,NaN,NaN,NaN
1,Access Board,10.0,11.0,11.0,11.0,11.0,12.0,12.0,12.0,13.0,13.0,13.0,CBO24,NaN,NaN,NaN,NaN,NaN,NaN
2,Administrative Conference of the United States,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,CBO24,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
#CBO21 
cbo_21 = pd.read_excel('../data/cbo-2021-02-11-spendingprojections.xlsx', header=7, sheet_name=1)
years_21 = [i for i in range(2021, 2032)]
cbo_21 = cbo_21[[col for col in cbo_21.columns if col not in years_21]].drop(columns=["Off-Budget?", "Unnamed: 9", "Unnamed: 21"]) #remove the BA cols, unnamed and unnecessary columns
cbo_21.columns = cbo_21.columns.str.replace('.1', '') #clean OT columns
#cbo_21["22-31"] = cbo_21.loc[:, '2022':'2031'].sum(axis=1) #sum the years
cbo_21["Agency"] = cbo_21["Agency"].str.strip()
cbo_21['Agency'] = cbo_21['Agency'].replace('Other Defense Civil Programs', 'Other Defense--Civil Programs').replace("Committee for Purchase from People Who Are Blind or Severely Dis", "Committee for Purchase From People Who Are Blind/Disabled" )
years_21 = [str(i) for i in years_21]
#cbo_21_agencies = cbo_21[["Agency"] + years_21 + ["22-31"]].groupby("Agency").sum().reset_index() #group by agency and sum the total
cbo_21_agencies = cbo_21[["Agency"] + years_21].groupby("Agency").sum().reset_index() #group by agency and sum the total
cbo_21_agencies["source"] = "CBO21"
cbo_21_agencies[["2018", "2019", "2020", "2032", "2033", "2034"]] = np.nan
cbo_21_agencies.head(3)

,Agency,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,source,2018,2019,2020,2032,2033,2034
0,Access Board,9,9,10,10,10,10,11,11,11,12,12,CBO21,NaN,NaN,NaN,NaN,NaN,NaN
1,Administrative Conference of the United States,3,3,3,3,3,3,4,4,4,4,4,CBO21,NaN,NaN,NaN,NaN,NaN,NaN
2,Advisory Council on Historic Preservation,7,7,7,8,8,8,8,9,9,9,9,CBO21,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
for agency in cbo_24_agencies["Agency"]:
    if agency not in cbo_21_agencies["Agency"].values:
        print(agency)

400 Years of African-American History Commission
Southeast Crescent Regional Commission
Southwest Border Regional Commission


In [67]:
#actual outlays (OMB)
omb = pd.read_excel('../data/omb_ots_data.xlsx', header=2, sheet_name="Table")
actual_yrs = [str(i) for i in range(2018, 2024)]
omb = omb[["Department or other unit"] + actual_yrs]
omb.rename(columns={"Department or other unit": "Agency"}, inplace=True)
omb['source'] = 'OMB'
omb[[str(year) for year in range(2024, 2035)]] = np.nan
omb.head(3)
omb["Agency"].unique()

array(['Legislative Branch', 'Judicial Branch',
       'Department of Agriculture', 'Department of Commerce',
       'Department of Defense--Military Programs',
       'Department of Education', 'Department of Energy',
       'Department of Health and Human Services',
       'Department of Homeland Security',
       'Department of Housing and Urban Development',
       'Department of the Interior', 'Department of Justice',
       'Department of Labor', 'Department of State',
       'Department of Transportation', 'Department of the Treasury',
       'Department of Veterans Affairs',
       'Corps of Engineers--Civil Works', 'Other Defense--Civil Programs',
       'Environmental Protection Agency',
       'Executive Office of the President',
       'General Services Administration',
       'International Assistance Programs',
       'National Aeronautics and Space Administration',
       'National Science Foundation', 'Office of Personnel Management',
       'Small Business Administrati

In [44]:
#omb[omb["Department or other unit"].str.contains("Social")]
omb[omb['Agency'] == 'Total outlays']

,Agency,2018,2019,2020,2021,2022,2023,source
35,Total outlays,4108981,4446952,6553620,6822461,6273259,6134672,OMB


In [69]:
# Reshape each dataframe to long format
cbo21_long = cbo_21_agencies.melt(id_vars=['Agency', 'source'], var_name='year', value_name='outlays')
cbo24_long = cbo_24_agencies.melt(id_vars=['Agency', 'source'], var_name='year', value_name='outlays')
omb_long = omb.melt(id_vars=['Agency', 'source'], var_name='year', value_name='outlays')

# Concatenate the reshaped dataframes
df_long = pd.concat([cbo24_long, cbo21_long, omb_long], ignore_index=True).rename(columns={"Agency": "agency"})
df_long

,agency,source,year,outlays
0,400 Years of African-American History Commission,CBO24,2024,3.0
1,Access Board,CBO24,2024,10.0
2,Administrative Conference of the United States,CBO24,2024,3.0
3,Advisory Council on Historic Preservation,CBO24,2024,9.0
4,Affordable Housing Program,CBO24,2024,248.0
...,...,...,...,...
5129,Allowances,OMB,2034,NaN
5130,Undistributed Offsetting Receipts,OMB,2034,NaN
5131,(On-budget),OMB,2034,NaN
5132,(Off-budget),OMB,2034,NaN
